<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Método entropía cruzada</center></span>

<center>Naive frozen lake</center>

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

5. Oleg Jarma, ojarmam@unal.edu.co 
6. Laura Lizarazo, ljlizarazore@unal.edu.co 

## <span style="color:blue">Referencias</span>

1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Richard S. Sutton, Andrew G. Barto, Reinforcement learning: an introduction, 2nd edition, 2020](http://library.lol/main/6502B74CE247C4CD4D4FB54747AD7C7E)
1. [Praveen Palanisamy - Hands-On Intelligent Agents with OpenAI Gym_ Your Guide to Developing AI Agents Using Deep Reinforcement Learning, 2020](http://library.lol/main/E4FD128CF9B93E0F7A542B053330517A)
1. [De Boer, P. and Kroese D.,  A Tutorial on the Cross-Entropy Method](http://web.mit.edu/6.454/www/www_fall_2003/gew/CEtutorial.pdf).

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [La API OpenAI Gym](#La-API-OpenAI-Gym)
* [El ambiente FrozenLake](#El-ambiente-FrozenLake)
* [Modificación del espacio de observaciones de FrozenLake](#Modificación-del-espacio-de-observaciones-de-FrozenLake)
* [El método de entropía cruzada en FrozenLake](#El-método-de-entropía-cruzada-en-FrozenLake)
* [El método se la entropía cruzada](#El-método-de-la-entropía-cruzada)
* [Modificación para el ambiente FrozenLake](#Modificación-para-el-ambiente-FrozenLake)

## <span style="color:blue">Introducción</span>

En esta lección vamos a usar el ambiente frozen lake para entrenar una agente con la técnica de entopía cruzada como hicimos con el ambiente CartPole. 

Vamos a modificar el espacio de observaciones mediante el uso de la clase *gym.ObservationWrapper*, de tal manera que nuestro código de CarPole pueda ser reutilizdo.

Lo que vamos a darnos cuenta es que el método de entropía cruzada no funciona en este caso. Est sistuacipon nos conducirá a implementar una solicipón mejorada del problema.

## <span style="color:blue">La API OpenAI Gym</span>

La API `OpenAI Gym` tiene una rica colección de ambientes para experimentos de aprendizaje reforzado (AR), usando una interfaz unificada.

La clase principal de Gym es *Env* (de environment). Sus métodos y atributos proveen la información necesaria para poder entrenar agentes que interactúen con el medio ambientes. Las piezas más importante disponibles con *Env* son:


<figure>
<img src="../Imagenes/environment.png" width="600" height="600" align="right"/>
</figure> 

- Un conjunto de `acciones` que se permite sean ejecutadas en el ambiente.
- El tamaño y bordes de las `observaciones` que el ambiente le provee al agente.
- Un método *step* para ejecutar un acción. El método regresa la nueva observación, la `recompensa` y la indicación de si el `episodio` ha terminado (*done*).
- Un método *reset* que retorna al ambiente a su *estado inicial* y entrega la primera observación.

Fuente: Alvaro Montenegro

### El espacio de acciones

Las acciones que puede realizar un agente puede ser discretas, continuas o incluso una combinación de ambas. Por ejemplo en un automóvil manejado de manera autónoma puede ser posible oprimir varios botones a la vez (discreto), oprimir el pedal del acelerador o del freno (continuo) y así. El espacio de acciones contiene o modela todas las posibilidades de acción del agente.

### El espacio de observaciones

Las observaciones son piezas de información que el ambiente puede entregar al agente en cualquier momento del tiempo, además de la recompensa. Las observaciones pueden ser muy simples como un arreglo unidimensional de números o  complejos tensores, como las imágenes provenientes de varias cámaras del automóvil.

Es claro que acciones y observaciones tienen similaridades, por lo que Gym dispone de una clase abstracta llamada *Space* que permite derivar espacios de acciones y espacios de observaciones. La siguiente imagen muestra una diagrama de la clase *Space*.

Existen otras subclases de *Space*, pero estas son las más comúnmente utilizadas.

<figure>
<center>
<img src="../Imagenes/jerarquia_clase_Space_Gym.jpeg" width="300" height="300" align="center"/>
</center>
</figure>

Jerarquía de la clase Space de OpenAI Gym. Fuente: [Maxim Lapan](http://library.lol/main/2E612EDEF1D325B87C7F5B1FB5542964)

La clase abstracta *Space* incluye dos métodos relevantes:

* *sample()*: retorna una muestra aleatoria del espacio.
* *contains(x)*: chequea si el argumento es un elemento del dominio del espacio.

Ambos métodos son abstractos y deben ser implementados en las subclases de *Space*.

* La clase *Discrete* representa un conjunto de ítems mutuamente excluyentes, numerados entre 0 y $n-1$. Su único atributo *n*, es una cuenta de los ítems que la clase describe. Por ejemplo. *Discrete(n=4)* puede representar por ejemplo un espacio de acciones de cuatro direcciones para moverse: *[left, right, up, down]*.

* La clase *Box* representa un tensor *n*-dimensional de números racionales con intervalos *[low, high]*. Por ejemplo puede representar el pedal de un acelerador con un valor simple entre 0.0 y 1.0. En pedal sería codificado en tal caso como *Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32)*. Otro ejemplo puede ser la observación de la pantalla en un juego Atari, la cual es RGB de tamaño 210*160: *Box(low=0, high=255, shape=(210, 160, 3), dtype=np.uint8)*.

* La última subclase de *Space* es *Tuple*, la cual permite combinar varias instancias de *Space* juntas. esta subclase permite crear espacios de acciones y observaciones de la complejidad que se desee.

Un ejemplo del uso de tupla puede ser el siguiente. Supongamos que deseamos modelar algunos controles de un automóvil. Para empezar la dirección (el ángulo de la dirección, en realidad), el acelerador y el freno pueder cambiar en cada instante de tiempo. Por otro lado, podemos tener botones discretos para representar la luces direccionales *(apagadas, izquierda, derecha)* y la bocina *(encendida, apagada)*. Entonces, podemos crear *Tupla(spaces=(Box(low=0.0, high=1.0, shape=(3,), dtype=np.float32), Discrete(3), Discrete(2)
)*.

### El ambiente

El ambiente es representado por la clase *Env*. Todo ambiente tiene dos miembros de tipo *Space*: 

* *action_space*: acciones permitidas que entrega el ambiente al agente.
* *observation_space*: observaciones proveidas por el ambiente al agente.

Además el ambiente tiene los siguientes miembros:

* *reset()*: coloca el ambiente en el estado inicial.
* *step()*: Permite al agente tomar una acción y retorna información como respuesta a la acción: la siguiente observación, la recompensa y la bandera indicando si el episodio ha  finalizado. Esta es la pieza central de la funcionalidad del ambiente.

Estas características permiten crear código genérico que podría trabajar con cualquier ambiente.

## <span style="color:blue">El ambiente FrozenLake</span>

Fuente: Alvaro Montenegro
<figure>
<center>
<img src="../Imagenes/frozenlake.png" width="400" height="400" align="left"/>
</center>
</figure>
    
La imagen es una representación del ambiente FrozenLake de Gym. En este ambiente, hay 16 posibles posiciones, por lo que el conjunto de observaciones es discreto con 16 valores. El agente siempre comienza en la parte superior izquierda y el objetivo es llegar a la parte inferior derecha.

Los sitios marcado con una X son huecos. Un episodio termina si el agente cae en un hueco o si llega al objetivo. En el primer caso (hueco) el agente recibe recompensa cero (0) y en el segundo caso (meta) el agente recibe recompensa uno (1). No importa cuanto se tarde en llegar ya sea a un hueco o a la meta, la recompensa siempre es la misma, es decir, 0 o 1.

Las acciones posible del agento son: arriba (up), abajo (down), izquierda (left) y derecha (right). Observe además que según en donde se encuentre el agente, no todas las acciones son posible en el siguiente paso del agente.

Este ambiente resulta más complicado que CartPole, debido al esquema de recompensa que entrega el ambiente. Una complicación adicional se introduce. Se va a suponer que el lago es resbaladizo. Esto implica que por ejemplo, el agente puede seleccionar izquierda, pero con un probabilidad almacenada en el ambiente puede resultar a la derecha (33.3\%), arriba (33.3\%) o abajo (33.3\%).

Lo primero que vamos  a hacer es modificar el espacio de observaciones a una codificación del tipo one-hot, con el propósito de reutilizar la implementación de CartPole que hicimos antes.

## <span style="color:blue">Modificación del espacio de observaciones de FrozenLake</span>

Primero demos una mirada al espacio de observaciones del ambiente. El método render de un objeto de tipo Env renderiza el espacio de acciones. Para interpretrar *S* es start, *F* es free, *H*  es hole y *G* es goal.

In [3]:
import gym
e = gym.make("FrozenLake-v1")
print(e.observation_space)
e.reset()
e.render()

Discrete(16)

SFFF
FHFH
FFFH
HFFG


### importa librerías

In [4]:
import gym, gym.spaces
from collections import namedtuple
import numpy as np

from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.optim as optim

### variables globales

In [5]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70
MAX_ITER = 500

### Envuelve el espacio de acciones

Con esta clase se envuelve el epacio de acciones para convertirlo en un espacio de tipo de tal manera que sea compatle con el tipo de espacio de CartPole. El nuevo tipo de observation será de  *Box* y contendrá un vector de tamaño 16, de tipo *onehot. 

In [13]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        shape = (env.observation_space.n, )
        self.observation_space = gym.spaces.Box(
            0.0, 1.0, shape=shape, dtype=np.float32)
        
    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res
    

Hicimos esto para poder reultizar el resto de la construcción de CartPole. Para la explicaciones consulte la lección [Entropía cruzada CartPole](ar_Metodo_Entropia_Cruzada.ipynb).

In [8]:
# red neuronal
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

# namedtuples para episodios y paso de episodio
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

### Entrenamiento

In [ ]:

if __name__ == "__main__":
    env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1"))
    # env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-frozenlake-naive")
    
    
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 0.8:
            print("Resuelto!")
            break
        if iter_no == max_iter:
            print("Terminado por máximo número de iteraciones. No resuelto")
            break
    writer.close()



### Abre tensorboard

In [23]:
!tensorboard --logdir=runs


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [16]:
# Cierra el ambiente. LA venta X11 se cierra.
env.close()

## <span style="color:blue">El método de la entropía cruzada</span>

Como se puede observar, en el ejemplo del ambiente FrozenLake el algoritmo termina por número máximo de iteraciones. Vamos a revisar la razón por la cual el método no funciona en este caso. En la siguiente sección se introducen algunas modificaciones que harpam que método converja. Más adelante retomaremos este ambiente con otro tipo de entrenamiento.  En la siguiente imagen se observan respectivamente la distribución de la recompensa para los casos de CartPole y FrozenLake.

Vamos a describir los detalles mínimos del métdo de entropía cruzada. Para obtenr información ampliada del método puede consultar [A Tutorial on the Cross-Entropy Method](http://web.mit.edu/6.454/www/www_fall_2003/gew/CEtutorial.pdf).

Fuente: [Maxim Lapan](http://library.lol/main/2E612EDEF1D325B87C7F5B1FB5542964)
<figure>
<center>
<img src="../Imagenes/distribucion_Reward_CartPole.png" width="400" height="400" align="left"/>
<img src="../Imagenes/distribucion_Reward_FrozenLake.png" width="400" height="400" align="right"/>
</center>
</figure>

Debido a que en el caso de CartPole, la recompensa es entregada por cada paso en cada episodio. Cada episodio entrega valores de recompensa que permite tener una distribución, que puede verse de forma aproximada como normal. No lo es en realidad, pero ciertamente hay valores diferentes para cada episodio, que dan cuenta de como estuvo el agente en el episodio. Por otro lado, FrozenLake solamente entrega dos tipos  de recompensa por episodio: uno o cero, según el agente haya arribado a la meta o no. 


<figure>
<center>
<img src="../Imagenes/probility_space.png" width="500" height="500" align="right"/>
</center>
</figure>

El objetivo del algoritmo es estimar la distribución de las acciones en el espacio de las observaciones. Si se dispone de una muestra de la distribución, la estimación es posible mediante en entrenamiento de una red neuronal. Eso es lo que lo se hace exáctamente cuando se entrena una red neuronal de clasificación.

Fuente: Alvaro Montenegro

El problema que tenemos en aprendizaje reforzado es que tal muestra no está disponible incialmente. Entonces, el método de entropía cruzada (MEC) provee un procedimiento estacástico para resolver el problema.

El MEC se basa en que en cada paso del algoritmo se obtiene una aproximación nejorada de la distribución de tal manera que cda vez se obtienen mejores muestras, en el sentido que la distribución aproximante se parece cada vez más a la distribución de las acciones.

La distribución aproximante es calculada por la red neuronal. El propósito de tomar los episodios *élite* por supuesto es mantener aquellos episodios que entregan mayor recompensa. 

El MEC funciona en CarPole, porque los episodios más largos son aquellos en donde el agente usa mejor la información de las observaciones para evitar que la viga caiga. 

El problema en el ambiente frozeLake, es que la recompensa de cada episodio es 1 o 0, sin importar que sucedió en cada paso del episodio. Como ilustra la imagen de la distribución en este caso. Entonces, no hay realmente forma de obtener episodios élite.

## <span style="color:blue">Modificación para el  ambiente FrozenLake</span>

Más adelante en este curso volveremos a este ambiente para resolver las limitaciones del MEC con otros métodos de parendizaje reforzado.

De momento haremos unas mejoras que ayuden al MEC a estimar la distribución de las acciones en el espacio de las observaciones. Haremos los siguiente:

* Lotes de episodios más largos. Pasaremos a 100 episodios por lote.
* Aplicaremos  el factor de descuento $\gamma$ a la recompensa. Asi espisodios más largos tendrán una menor recompensa y viceversa.Esto incrementa la variabilidad de la distribución de la recompensa.
* Mantendremos episodios élite por más largo tiempo.
* Decreceremos la rata de aprendizaje. Esto implica que la red neuronal tendra más tiemponpara ver en promedio mas muestras de entrenamiento.
* Mucho mayor tiempo de entrenamiento.

Por favor revise, corra y modifique si lo considera necesario, el siguiente código.


In [ ]:
import random
import gym
import gym.spaces
from collections import namedtuple
import numpy as np

from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 100
PERCENTILE = 30
GAMMA = 0.9
MAX_ITER = 1000


class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res


class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch, percentile):
    filter_fun = lambda s: s.reward * (GAMMA ** len(s.steps))
    disc_rewards = list(map(filter_fun, batch))
    reward_bound = np.percentile(disc_rewards, percentile)

    train_obs = []
    train_act = []
    elite_batch = []
    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation,
                                 example.steps))
            train_act.extend(map(lambda step: step.action,
                                 example.steps))
            elite_batch.append(example)

    return elite_batch, train_obs, train_act, reward_bound


if __name__ == "__main__":
    random.seed(12345)
    env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1"))
    # env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.001)
    writer = SummaryWriter(comment="-frozenlake-ajustado)

    full_batch = []
    for iter_no, batch in enumerate(iterate_batches(
            env, net, BATCH_SIZE)):
        reward_mean = float(np.mean(list(map(
            lambda s: s.reward, batch))))
        full_batch, obs, acts, reward_bound = \
            filter_batch(full_batch + batch, PERCENTILE)
        if not full_batch:
            continue
        obs_v = torch.FloatTensor(obs)
        acts_v = torch.LongTensor(acts)
        full_batch = full_batch[-500:]

        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, rw_mean=%.3f, "
              "rw_bound=%.3f, batch=%d" % (
            iter_no, loss_v.item(), reward_mean,
            reward_bound, len(full_batch)))
        writer.add_scalar("pérdida", loss_v.item(), iter_no)
        writer.add_scalar("recompensa_promedio", reward_mean, iter_no)
        writer.add_scalar("recompensa_frontera", reward_bound, iter_no)
        if reward_mean > 0.8:
            print("Resuelto!")
            break
            if iter_no == MAX_ITER:
            print("Terminado por máximo número de iteraciones. No resuelto")
            break
    writer.close()


### Abre tensorboard

In [ ]:
!tensorboard --logdir=runs

In [16]:
# Cierra el ambiente. LA venta X11 se cierra.
env.close()